## Anonmymizing the customer_information.csv and calculating the k-anonymity of the new dataset

Importing necessary packages

In [10]:
import pandas as pd
import numpy as np
import hashlib
import re
import os
import math
from geopy.geocoders import Nominatim
from functions import *

Loading the original dataset

In [11]:
# Read in data to be anonymised
data = pd.read_csv("Data/customer_information.csv")

# Create anon_data variable as initial data with unneeded direct identifiers dropped
#anon_data = data.drop(['given_name', 'surname', 'phone_number', 'national_insurance_number', 'bank_account_number'], axis=1)
anon_data = pd.DataFrame()

In [12]:
# Clean NIN formatting and assign Sample ID as a hashed form of the NIN
key = os.urandom(16)
data["national_insurance_number"], anon_data['Sample.ID'], salts = zip(*data["national_insurance_number"].apply(
    lambda x: hash(key, re.sub(r'(.{2})(?!$)','\\1 ', x.replace(' ', '') ))))

In [13]:
# Create a reference table between NIN and respective hashed NIN
reference_table = pd.DataFrame()
reference_table['Hashed.NIN'] = anon_data['Sample.ID']
reference_table['Salt'] = salts
reference_table['Key'] = key.hex()
reference_table['NIN'] = data['national_insurance_number']

In [14]:
# Assign gender
anon_data['Gender'] = data['gender']

# Assign birthdate as banded birthyears
# Select the birth year only
birthyears = pd.DatetimeIndex(data['birthdate']).year
# Band the birth years into 5-year intervals
anon_data['Birthyear'] = pd.cut(birthyears, np.arange(birthyears.min(), birthyears.max()+20, 20), right=False)

# Assign postcode as truncated postcode
anon_data['Postcode'] = data['postcode'].apply(lambda x: re.search('[a-zA-Z]*', x).group(0))

#anon_data['Region'] = data['postcode'].apply(lambda x: postcode_to_region(x))
postcode_dictionary = pd.read_csv('Data/postcode_region.csv')
anon_data = pd.merge(anon_data, postcode_dictionary.iloc[: , :-1], on='Postcode', how='left')
#anon_data['Region'] = anon_data['Postcode'].to_frame().merge(postcode_dictionary.iloc[: , :-1], on='Postcode', how='left')

In [15]:
# Assign weight and height as banded weights and heights
#anon_data['Weight'] = pd.cut(data['weight'], np.arange(math.floor(data['weight'].min()), math.floor(data['weight'].max()+20), 20), right=False)
weight_noise = np.random.normal(0,1,1000)*5
anon_data['Weight'] = round(data['weight']+weight_noise, 1)


In [16]:
# Round minimum and maximum heights to nearest one-fifth prior to making bins
#anon_data['Height'] = pd.cut(data['height'], np.arange(round(data['height'].min()*5)/5, (round(data['height'].max()*5)/5)+0.2, 0.2), right=False)
height_noise = np.random.normal(0,1,1000)/5
anon_data['Height'] = round(data['height']+height_noise, 2)
bmi = data['weight'] / data['height']**2
print(anon_data['Height'])
print(data['height'])#anon_data['BMI'] = pd.cut(bmi, bins=[math.floor(bmi.min()), 18.5, 25, 30, round(bmi.max(), -1)], right=False)
#print(anon_data['BMI'])
# Assign avg_drinks as banded avg_drinks
#anon_data['avg_n_drinks_per_week'] = pd.cut(anon_data['avg_n_drinks_per_week'], np.linspace(0, 2, 9))


0      1.64
1      1.66
2      1.87
3      1.71
4      1.70
       ... 
995    1.72
996    1.69
997    1.93
998    1.36
999    1.94
Name: Height, Length: 1000, dtype: float64
0      1.73
1      1.74
2      1.88
3      1.56
4      1.81
       ... 
995    1.98
996    1.85
997    2.00
998    1.50
999    1.65
Name: height, Length: 1000, dtype: float64


In [17]:
import country_converter as coco
from geopy.geocoders import Nominatim


def parse_country(country_name):
    country = coco.convert(country_name, to='name_short', include_obsolete=True)
    if(country_name == 'Svalbard & Jan Mayen Islands'): # Hard-coded fix for the unmatched country
        country = 'Norway'
    return country

def country_to_hemisphere2(country_name):
    clean_country = parse_country(country_name)
    try:
        parsed_country = parse_country(country_name)
        if(parsed_country == "Micronesia"): # Hard-coded fix for territory, cannot match as countr
            hemisphere = "Southern Hemisphere"
        else:
            hemisphere = ("Southern" if Nominatim(user_agent="CDM").geocode((parsed_country)).latitude < 0 else "Northern") + " Hemisphere"
        print("Input:",country_name,", output:", hemisphere)
        #idx += 1
        return hemisphere
    except:
        print("Input:",country_name,", output:", "Error")
        return "Error"
    
# Assign education level as banded education level
anon_data['Education.Level'] = data['education_level'].map(lambda x: "Higher" if x in ["bachelor", "masters", "phD"] else "BasicOther")

anon_data['Location.of.Birth'] = data['country_of_birth'].apply(lambda x: country_to_hemisphere2(x))

# Output the files
anon_data.to_csv("output.csv", sep=",", index=None)
reference_table.to_csv("reference_table.csv", sep=",", index=None)

Input: Armenia , output: Northern Hemisphere
Input: Northern Mariana Islands , output: Northern Hemisphere
Input: Venezuela , output: Northern Hemisphere
Input: Eritrea , output: Northern Hemisphere
Input: Ecuador , output: Southern Hemisphere
Input: Argentina , output: Southern Hemisphere
Input: Cook Islands , output: Southern Hemisphere
Input: Saint Vincent and the Grenadines , output: Northern Hemisphere
Input: Kiribati , output: Northern Hemisphere
Input: Belize , output: Northern Hemisphere
Input: Tokelau , output: Southern Hemisphere
Input: French Guiana , output: Northern Hemisphere
Input: Saint Lucia , output: Northern Hemisphere
Input: Congo , output: Northern Hemisphere
Input: Korea , output: Northern Hemisphere
Input: Syrian Arab Republic , output: Northern Hemisphere
Input: American Samoa , output: Southern Hemisphere
Input: Palestinian Territory , output: Northern Hemisphere
Input: Saint Barthelemy , output: Northern Hemisphere
Input: New Zealand , output: Southern Hemisph

In [18]:
anon_data


,Sample.ID,Gender,Birthyear,Postcode,Region,Region.1,Weight,Height,Education.Level,Location.of.Birth
0,5ddd6f2db40d9b1366913deed67bcafc197e97248fd25d...,F,"[1975, 1995)",LS,England,North East,76.0,1.64,Higher,Northern Hemisphere
1,b012ae4fe956e7b18cc2b1cb4fd115af3ce52af810339a...,M,"[1995, 2015)",M,England,North West,71.4,1.66,BasicOther,Northern Hemisphere
2,bf3f3936bfa3597dfd7564f83a596fba8207d56f10bc5e...,F,"[1975, 1995)",SO,England,South East,96.5,1.87,Higher,Northern Hemisphere
3,615bd73151e8e2ca31663b76c45bef88c2c7d0917b9e81...,F,"[1995, 2015)",B,England,West Midlands,61.5,1.71,BasicOther,Northern Hemisphere
4,f43ebbd3705328603e7ec577db42abe3314684bdf2e0c1...,F,"[1955, 1975)",TQ,England,South West,97.8,1.70,BasicOther,Southern Hemisphere
...,...,...,...,...,...,...,...,...,...,...
995,e30aebf3a70904e746af797cc6b612efecd3d0f85e7ef1...,M,"[1955, 1975)",SA,Other,Wales,93.9,1.72,BasicOther,Northern Hemisphere
996,da418e86e860869773819bdafbdbce97f243a987ac182d...,M,"[1995, 2015)",TS,England,North East,58.2,1.69,BasicOther,Northern Hemisphere
997,2fbf5c1205cf79fe2593e4e70d25566e7820990a7daffb...,F,"[1955, 1975)",G,Other,Scotland,96.6,1.93,BasicOther,Southern Hemisphere
998,7b8f47d3ba15b50359511224ffa2bc2e8d52291523177b...,F,"[1955, 1975)",CT,England,South East,79.1,1.36,Higher,Northern Hemisphere


In [19]:
# Calculating K-anonymity, 2 anonymous dataset
k = 2
############
# Checking k-anonymity, fix this for variable names
df_count = anon_data.groupby(['Gender', 'Birthyear', 'Education.Level', 'Region', 'Location.of.Birth']).size().reset_index(name = 'Count') 
print(df_count)
print(df_count[df_count['Count']==1])
# print(df_count.size().describe())

# print(anon_data.groupby('Education.Level').size())
# print(anon_data.groupby('Continent.of.Birth').size())
# print(anon_data.groupby('Region').size())

   Gender     Birthyear Education.Level   Region    Location.of.Birth  Count
0       F  [1955, 1975)      BasicOther  England                Error      0
1       F  [1955, 1975)      BasicOther  England  Northern Hemisphere     77
2       F  [1955, 1975)      BasicOther  England  Southern Hemisphere     37
3       F  [1955, 1975)      BasicOther    Other                Error      0
4       F  [1955, 1975)      BasicOther    Other  Northern Hemisphere     12
..    ...           ...             ...      ...                  ...    ...
67      M  [1995, 2015)          Higher  England  Northern Hemisphere     19
68      M  [1995, 2015)          Higher  England  Southern Hemisphere      5
69      M  [1995, 2015)          Higher    Other                Error      0
70      M  [1995, 2015)          Higher    Other  Northern Hemisphere      5
71      M  [1995, 2015)          Higher    Other  Southern Hemisphere      2

[72 rows x 6 columns]
   Gender     Birthyear Education.Level   Region Loca